## 150 Weighted Mean @month_lag

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
from mv_wg_avg import exp_weight_avg


os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [15]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00, 180.50it/s]


SystemExit: 

In [3]:
df = df_new
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
# df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'month_lag', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_new = df.copy()
del df
gc.collect()

df = df_hist
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
# df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'month_lag', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

,card_id,month_lag,authorized_flag,category_1,purchase_amount,installments,diff_days
19095896,C_ID_00007093c1,-12,1,0,0.037176,2,NaN
19095775,C_ID_00007093c1,-12,1,0,-0.467415,1,0.0
19095845,C_ID_00007093c1,-12,1,0,-0.572600,1,1.0
19095866,C_ID_00007093c1,-12,1,0,-0.331575,3,3.0
19095808,C_ID_00007093c1,-11,1,0,-0.483944,1,10.0


In [16]:
df_trans = pd.concat([df_hist, df_new], axis=0)
df_trans = reduce_mem_usage(df_trans)
df_trans['purchase_amount'] -=  df_trans['purchase_amount'].min() + 1.0e-4
auth1 = df_trans[df_trans['authorized_flag']==1]
auth0 = df_trans[df_trans['authorized_flag']==0]
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)

### month_lag levelでgroupbyして、加重平均を行う

In [18]:
# auth1
df = auth1.copy()
fname = 'auth1'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'sum']
aggs['purchase_amount'] = ['mean', 'sum', 'max', 'min']
aggs['diff_days'] = ['mean']
aggs['installments'] = ['mean']
aggs['record'] = ['sum']

# 値が大きいほど、直近の数字となる -> 逆にしなきゃ
df['month_lag'] =  df['month_lag'] - df['month_lag'].min()
tmp = df.groupby([key, 'month_lag']).agg(aggs)
new_columns = get_new_columns(fname, aggs)
tmp.columns = new_columns

tmp  = reduce_mem_usage(tmp).reset_index()

label = 'month_lag'
weight_list = []
ignore_cols = [key, label, 'tmp', 'weight', 'diff']
base = tmp[[key, label]].drop_duplicates()
base['tmp'] = 0
base = base.set_index([key, label]).unstack()
base = base.fillna(0).stack().reset_index()[[key, label]]
tmp = base.merge(tmp, how='left', on=[key, label]).fillna(0)

for col in tqdm(tmp.columns):
    if col in ignore_cols:continue
    tmp_weight = exp_weight_avg(data=tmp, level=key, value=col, weight=0.9, label=label)
    weight_list.append(tmp_weight)
    
df_auth1_wg = pd.concat(weight_list, axis=1)
df_auth1_wg.head()

Mem. usage decreased to 68.25 Mb (63.8% reduction)


100%|██████████| 11/11 [00:23<00:00,  2.53s/it]


,auth1_category_1_mean_wg0.9_avg@card_id,auth1_category_1_sum_wg0.9_avg@card_id,auth1_purchase_amount_mean_wg0.9_avg@card_id,auth1_purchase_amount_sum_wg0.9_avg@card_id,auth1_purchase_amount_max_wg0.9_avg@card_id,auth1_purchase_amount_min_wg0.9_avg@card_id,auth1_diff_days_mean_wg0.9_avg@card_id,auth1_installments_mean_wg0.9_avg@card_id,auth1_record_sum_wg0.9_avg@card_id
card_id,,,,,,,,,
C_ID_00007093c1,0.175170,1.509114,0.163501,1.327832,0.463987,0.037090,2.834955,1.170488,6.810740
C_ID_0001238066,0.029504,0.387928,0.110141,2.112173,0.414063,0.011890,0.962945,1.203898,12.916592
C_ID_0001506ef0,0.000000,0.000000,0.256258,1.057207,0.855813,0.016769,7.324482,0.000000,4.199974
C_ID_0001793786,0.000000,0.000000,0.612185,10.826265,2.670048,0.060199,1.277419,0.000000,17.196844
C_ID_000183fdda,0.017341,0.314078,0.167742,3.046766,1.045139,0.017040,2.201996,1.193683,12.144639


In [19]:
# auth0
df = auth0.copy()
fname = 'auth0'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'sum']
aggs['purchase_amount'] = ['mean', 'sum', 'max', 'min']
aggs['diff_days'] = ['mean']
aggs['installments'] = ['mean']
aggs['record'] = ['sum']

# 値が大きいほど、直近の数字となる -> 逆にしなきゃ
df['month_lag'] =  df['month_lag'] - df['month_lag'].min()
tmp = df.groupby([key, 'month_lag']).agg(aggs)
new_columns = get_new_columns(fname, aggs)
tmp.columns = new_columns

tmp  = reduce_mem_usage(tmp).reset_index()

label = 'month_lag'
weight_list = []
ignore_cols = [key, label, 'tmp', 'weight', 'diff']
base = tmp[[key, label]].drop_duplicates()
base['tmp'] = 0
base = base.set_index([key, label]).unstack()
base = base.fillna(0).stack().reset_index()[[key, label]]
tmp = base.merge(tmp, how='left', on=[key, label]).fillna(0)

for col in tqdm(tmp.columns):
    if col in ignore_cols:continue
    tmp_weight = exp_weight_avg(data=tmp, level=key, value=col, weight=0.9, label=label)
    weight_list.append(tmp_weight)
    
df_auth0_wg = pd.concat(weight_list, axis=1)
df_auth0_wg.head()

Mem. usage decreased to 30.81 Mb (50.5% reduction)


100%|██████████| 11/11 [00:16<00:00,  1.68s/it]


,auth0_category_1_mean_wg0.9_avg@card_id,auth0_category_1_sum_wg0.9_avg@card_id,auth0_purchase_amount_mean_wg0.9_avg@card_id,auth0_purchase_amount_sum_wg0.9_avg@card_id,auth0_purchase_amount_max_wg0.9_avg@card_id,auth0_purchase_amount_min_wg0.9_avg@card_id,auth0_diff_days_mean_wg0.9_avg@card_id,auth0_installments_mean_wg0.9_avg@card_id,auth0_record_sum_wg0.9_avg@card_id
card_id,,,,,,,,,
C_ID_00007093c1,0.125839,0.286360,0.281106,0.798916,0.451573,0.141680,1.161957,1.081644,2.577805
C_ID_0001238066,0.000000,0.000000,0.035280,0.051069,0.035280,0.035280,0.315080,0.234689,0.339716
C_ID_0001506ef0,0.000000,0.000000,0.005551,0.009991,0.006086,0.005015,1.467515,0.064831,0.420168
C_ID_0001793786,0.052513,0.210054,0.176271,0.769777,0.327467,0.037360,0.594847,0.154812,2.487636
C_ID_000183fdda,0.000000,0.000000,0.051626,0.295961,0.077253,0.023576,0.317085,0.291796,0.498520


In [20]:
for df in [df_auth1_wg, df_auth0_wg]:
    df_train = df_train.join(df)
    df_test = df_test.join(df)
for col in df_train.columns:
    df_train.rename(columns={col:col.replace('.', '_')}, inplace=True)
    df_test.rename(columns={col:col.replace('.', '_')}, inplace=True)
print(df_train.shape)
display(df_train.head())

(201917, 23)


,first_active_month,feature_1,feature_2,feature_3,target,auth1_category_1_mean_wg0_9_avg@card_id,auth1_category_1_sum_wg0_9_avg@card_id,auth1_purchase_amount_mean_wg0_9_avg@card_id,auth1_purchase_amount_sum_wg0_9_avg@card_id,auth1_purchase_amount_max_wg0_9_avg@card_id,auth1_purchase_amount_min_wg0_9_avg@card_id,auth1_diff_days_mean_wg0_9_avg@card_id,auth1_installments_mean_wg0_9_avg@card_id,auth1_record_sum_wg0_9_avg@card_id,auth0_category_1_mean_wg0_9_avg@card_id,auth0_category_1_sum_wg0_9_avg@card_id,auth0_purchase_amount_mean_wg0_9_avg@card_id,auth0_purchase_amount_sum_wg0_9_avg@card_id,auth0_purchase_amount_max_wg0_9_avg@card_id,auth0_purchase_amount_min_wg0_9_avg@card_id,auth0_diff_days_mean_wg0_9_avg@card_id,auth0_installments_mean_wg0_9_avg@card_id,auth0_record_sum_wg0_9_avg@card_id
card_id,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,0.000000,0.000000,0.101293,2.430879,0.505027,0.013875,0.857381,0.000000,19.527450,0.000000,0.000000,0.049983,0.091829,0.065151,0.035640,1.011613,0.150492,1.068531
C_ID_3d0044924f,2017-01,4,1,0,0.392913,0.051964,1.349668,0.102519,2.368006,0.766222,0.009767,2.911306,1.366678,22.096358,0.072826,0.140887,0.230050,0.528447,0.384630,0.156320,0.692780,1.535690,0.923209
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,0.000000,0.000000,0.067480,0.158323,0.103184,0.036546,12.424061,0.000000,1.942653,0.000000,0.000000,0.006066,0.006066,0.006066,0.006066,0.109862,0.000000,0.081831
C_ID_186d6a6901,2017-09,4,3,0,0.142495,0.117714,1.128830,0.086117,0.679110,0.339482,0.012770,3.668151,0.717020,6.620854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,0.046375,1.230233,0.122049,3.199550,1.310408,0.007474,0.454810,0.631820,15.359728,0.129663,0.388988,1.048170,3.117201,1.056833,1.030845,0.000000,1.672648,0.622381


### Save feature

In [21]:
fname = '150_wgm'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('_wg')):continue
    length = len(df_train[col].dropna())
    if length/len(df_train)<0.1:
        print(col)
        continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)